# System B: TV Show Audiences

1. View 1: Genre Popularity
Visualization Technique: Horizontal Bar Chart
Purpose: Show the popularity of TV show genres based on audience ratings.
Brushing & Linking: Selection in this view will filter other views based on the selected genre.

2. View 2: Country Distribution
Visualization Technique: Choropleth Map
Purpose: Show the distribution of TV shows by country and count/popularity.
Brushing & Linking: Selection in this view will filter other views based on the selected country.

3. View 3: Season Duration
Visualization Technique: N/A
Purpose: Show the relation between the popularity of TV show and season durations.
Brushing & Linking: Selection in this view will filter other views based on the selected duration range.

4. View 4: Language Diversity
Visualization Technique: Pie Chart
Purpose: Show the distribution of TV shows by original language.
Brushing & Linking: Selection in this view will filter other views based on the selected language.

In [1]:
import altair as alt
import pandas as pd
from IPython.display import display

file_path = r'C:\Users\aadit\Downloads\Documents\Information Visualization\TMDB_tv_dataset_v3.csv'
cvd_data = pd.read_csv(file_path)

tv_shows_data = cvd_data
tv_shows_data.dropna(inplace=True)

tv_shows_data = tv_shows_data.sample(n=100,random_state=40)

alt.data_transformers.enable('json')
tv_shows_data.head(2)

,id,name,number_of_seasons,number_of_episodes,original_language,vote_count,vote_average,overview,adult,backdrop_path,...,tagline,genres,created_by,languages,networks,origin_country,spoken_languages,production_companies,production_countries,episode_run_time
885,73544,Warrior,3,30,en,460,8.027,"A gritty, action-packed crime drama set during...",False,/aXZmDoGywrHlv2OIvurlCiTXkXv.jpg,...,This isn't China. This is Chinatown.,"Crime, Drama, Action & Adventure",Jonathan Tropper,"en, zh","Cinemax, Max",US,"English, 普通话","Perfect Storm Entertainment, Bruce Lee Enterta...",United States of America,55
1245,63522,F is for Family,5,44,en,315,7.344,"Follow the Murphy family back to the 1970s, wh...",False,/bdntcURJ4dSr7zoLeyB6ZGrMynD.jpg,...,Kick 'em in the cul-de-sac.,"Comedy, Animation","Bill Burr, Michael Price",en,Netflix,US,English,"Wild West Television, Gaumont International Te...","France, United States of America",26


In [2]:
# View 1: Genre Popularity

genre_popularity_chart = alt.Chart(tv_shows_data).mark_bar().encode(
    y=alt.Y('genres:N', title='Genre'),
    x=alt.X('mean(vote_average):Q', title='Mean Vote Average'),
    color=alt.Color('genres:N', legend=None),
    tooltip=['genres:N', alt.Tooltip('mean(vote_average)', title='Mean Vote Average')]
).properties(
    title='Genre Popularity of TV Shows',
    width=500,
    height=1300 
).interactive()

display(genre_popularity_chart)

alt.Chart(...)

In [45]:
# View 2: Continent/Country Distribution by count and popularity


In [46]:
# View 3: Heatmap with Circle Size Encoding Popularity, Can also show top 10 shows 

In [44]:
# View 4: Language Diversity
language_diversity_chart = alt.Chart(tv_shows_data).mark_arc().encode(
    theta=alt.Theta('count():Q', title='Number of Shows'),
    color=alt.Color('original_language:N', legend=None),
    tooltip=['original_language:N', alt.Tooltip('count()', title='Number of Shows')]
).properties(
    title='Language Diversity of TV Shows',
    width=500,
    height=300 
).interactive()

display(language_diversity_chart)

alt.Chart(...)